# Wildfire risk by US county

## Introduction

### Obtain all zipcodes for the state

In [1]:
state_name = 'CA'
filename = 'data/zipcodes_'+state_name+'.csv'
import pandas as pd
counties = pd.read_csv(filename, names=['zipcode','city','county'])
print("%d zipcodes found..." % len(counties))
with pd.option_context('display.max_rows',10) : 
    print(counties)

2589 zipcodes found...
      zipcode              city       county
0       90001       Los Angeles  Los Angeles
1       90002       Los Angeles  Los Angeles
2       90003       Los Angeles  Los Angeles
3       90004       Los Angeles  Los Angeles
4       90005       Los Angeles  Los Angeles
...       ...               ...          ...
2584    96157  South Lake Tahoe    El Dorado
2585    96158  South Lake Tahoe    El Dorado
2586    96160           Truckee       Nevada
2587    96161           Truckee       Nevada
2588    96162           Truckee       Nevada

[2589 rows x 3 columns]


### Load cencus data 

In [2]:
from uszipcode import SearchEngine

search = SearchEngine(simple_zipcode=True)
countydata = {}
skipped_zip=0
for zipcode in list(counties.index) : 
    data = search.by_zipcode(str(zipcode)).to_dict()
    try : 
        countydata[zipcode] = {
            "City":data["major_city"]+" "+data["state"],
            "Latitude":data["lat"],
            "Longitude":data["lng"],
            "Population":data["population"],
            "Density":int(data["population_density"]),
            "Occupancy":int(data["occupied_housing_units"] / data["housing_units"] * 100)/100,
            "Value":int(data["median_home_value"]/100)/10,
            "Income":int(data["median_household_income"]/100)/10
        }
    except:
        skipped_zip = skipped_zip+1
#         print("WARNING: error processing %s rbs, skipping" % (zipcode))
print("WARNING: missing data in %d out of %d zipcodes" % (skipped_zip,len(counties)))

### Load FEMA data

The data is retrieved from https://www.fema.gov/openfema-dataset-disaster-declarations-summaries-v2

In [3]:
fema_filename = 'data/DisasterDeclarationsSummaries.csv'
fire_data = pd.read_csv(fema_filename, index_col=[1],names=["femaDeclarationString",
                                                            "disasterNumber",
                                                            "state",
                                                            "declarationType",
                                                            "declarationDate",
                                                            "fyDeclared",
                                                            "incidentType",
                                                            "declarationTitle",
                                                            "ihProgramDeclared",
                                                            "iaProgramDeclared",
                                                            "paProgramDeclared",
                                                            "hmProgramDeclared",
                                                            "incidentBeginDate",
                                                            "incidentEndDate",
                                                            "disasterCloseoutDate",
                                                            "fipsStateCode",
                                                            "fipsCountyCode",
                                                            "placeCode",
                                                            "designatedArea",
                                                            "declarationRequestNumber",
                                                            "hash","lastRefresh"])
print('Types of incidents filed in the data set...')
print(set(fire_data['incidentType']))
# for item in list(fire_data.index) : 
#     print(item)
# with pd.option_context('display.max_rows',10) : 
#     print(fire_data)

Types of incidents filed in the data set...
{'Tsunami', 'Hurricane', 'Toxic Substances', 'Fire', 'incidentType', 'Drought', 'Chemical', 'Tornado', 'Dam/Levee Break', 'Coastal Storm', 'Human Cause', 'Typhoon', 'Flood', 'Freezing', 'Other', 'Severe Ice Storm', 'Fishing Losses', 'Terrorist', 'Volcano', 'Earthquake', 'Severe Storm(s)', 'Snow', 'Mud/Landslide'}


/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,5,8,9,10,11,15,16,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


*Assuming only fire incidents are relevent to this study

### Getting California fire incident data

In [4]:
import requests 
from bs4 import BeautifulSoup
import re
table_content = []
final_list = []
list_of_avail_years = range(2013,2020,1)
for yr in list_of_avail_years : 
    url = 'https://www.fire.ca.gov/incidents/'+str(yr)+'/'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    html_content = {}
    for i in range(len(soup.find_all('div'))) : 
        try : 
            if soup.find_all('div')[i].attrs['class'] == ['section', 'incidents-esri-webmap']: 
                html_content = soup.find_all('div')[i].text
        except : 
            continue
    
    table_content=eval(html_content.partition('create_map([')[2].split("\n")[0][:-3])
    for t in table_content : 
        final_list.append(t)


In [5]:
fires_df = pd.DataFrame(final_list)

Example fire statistics on Ranch Fires

In [6]:
print(fires_df.loc[fires_df['Name'].str.contains('Ranch')])

      IncidentId                              UniqueId  \
87           740  591a56a5-e9c0-45a0-96bd-34098588c39b   
208          861  9487beeb-6c40-46fc-b4b3-a353a688a826   
323          976  a4e4603a-f0fc-4d7a-845b-6c668bfcb81e   
488         1141  5689b708-61e8-4451-8ad6-68d9684704c6   
735         1388  5da65d7f-cc75-4c0c-b221-1a1e3000a120   
834         1487  2fd950a8-fadb-467c-ab59-88e7ea007738   
1144        1796  b2adde70-a40e-4b81-9f5f-8f6b96042af2   
1253        1905  90b0daf7-5d84-42d9-bb35-bb78d4faf950   
1313        1965  20b57ee7-b0b7-47db-8a12-e5fcdb114459   
1424        2079  65330473-c21f-446b-a6f4-7bd76bf0ebdc   
1564        2315  e2a0252b-2da9-4967-ba33-7abe3ced1efa   
1620        2371  69d98a94-4980-4dc0-b388-61c37c635f32   

                                Name  \
87                       Rancho Fire   
208                       Ranch Fire   
323                       Ranch Fire   
488                      Rancho Fire   
735                       Ranch Fire   
834  

Sorting by county by year 

In [7]:
county_list = []
county_list_repeated = set(fires_df['CountiesList'].tolist())
while ("" in county_list_repeated) : 
    county_list_repeated.remove("")
for item in county_list_repeated : 
    item_list = []
    item_list = item.split(",")
    if len(item_list) > 1 : 
        for c in item_list : 
            county_list.append(c.strip())
    else : 
        county_list.append(item)
county_set = set(county_list)
print('List of counties affect by fires : \n', county_set)

List of counties affect by fires : 
 {'Yuba', 'Sierra', 'Modoc', 'Marin', 'Solano', 'Kern', 'Shasta', 'Ventura', 'Mono', 'Tehama', 'Butte', 'El Dorado', 'Fresno', 'Santa Barbara', 'Napa', 'State of Oregon', 'San Luis Obispo', 'Kings', 'Trinity', 'San Diego', 'Nevada', 'Amador', 'Sacramento', 'Placer', 'Plumas', 'Riverside', 'Santa Clara', 'Contra Costa', 'Orange', 'Mariposa', 'Sonoma', 'Stanislaus', 'Glenn', 'Monterey', 'Madera', 'Alameda', 'Lake', 'San Mateo', 'San Bernardino', 'Tulare', 'Inyo', 'Mendocino', 'Sutter', 'Lassen', 'San Joaquin', 'Calaveras', 'San Benito', 'Alpine', 'Humboldt', 'Mexico', 'Los Angeles', 'State of Nevada', 'Del Norte', 'Siskiyou', 'Colusa', 'Merced', 'Santa Cruz', 'Yolo', 'Tuolumne'}


In [8]:
print("Total of %d out 58 total counties in CA affected by fires" % len(county_set))

Total of 59 out 58 total counties in CA affected by fires


The county size information was acquired from https://www.counties.org/pod/square-mileage-county on Nov 15, 2019

In [21]:
county_size = pd.read_csv('data/counties_size.csv', names=['county','size'])
county_size.set_index('county',inplace=True)
print(county_size.loc['Inyo'])
print("%d counties found..." % len(county_size))
with pd.option_context('display.max_rows',10) : 
    print(county_size)

size    10181
Name: Inyo, dtype: int64
58 counties found...
                 size
county               
San Bernardino  20057
Inyo            10181
Kern             8132
Riverside        7206
Siskiyou         6278
...               ...
Amador            595
Marin             520
San Mateo         448
Santa Cruz        445
San Francisco      47

[58 rows x 1 columns]


Note, the area is quoted in square miles.

Checking that the county is listed in the manifest : 

In [10]:
county_manifest = county_size['county'].values.tolist()
county_exclude = []
for county in county_set : 
    if county not in county_manifest : 
        print("ERROR: Unknown County %s in fire data" % county)
        county_exclude.append(county)

ERROR: Unknown County State of Oregon in fire data
ERROR: Unknown County Mexico in fire data
ERROR: Unknown County State of Nevada in fire data


In [53]:
# Summary of required variables
# county_set - counties that had records of fires 
# fires_df - dataframe will all the fire incidents in CA 
# county_size - size of county for multiple counties that were affected by the same fire

county_acres_burnt = {}

for county in county_set : 
    if county in county_manifest :
        county_acres_burnt[county]=0
        for row in fires_df.itertuples(index = True, name ='Pandas'): 
            if county in getattr(row, "CountiesList") : 
                county_temp = [x.strip(' ') for x in (getattr(row, "CountiesList")).split(",")]
                # Look up the square footage of each county 
                if getattr(row,"AcresBurnedDisplay")!='' :
                    total_acres = float(getattr(row,"AcresBurnedDisplay").replace(',', ''))

                if len(county_temp) > 1:
                    total_area = 0
                    c_dict = {}
                    for c in county_temp :
                        if c not in county_manifest :
                            print("WARNING: county/region %s is not in CA, excluding from analysis" % c)
                        else :
                            total_area = total_area+county_size.loc[c]['size']
                            c_dict[c]=county_size.loc[c]['size']
                    for key,elem in c_dict.items() : 
                        if key not in county_acres_burnt : 
                            county_acres_burnt[key]=float(total_acres)*float(elem)/total_area
                        else : 
                            county_acres_burnt[key]=county_acres_burnt[key]+float(total_acres)*float(elem)/total_area
                else : 
                    if getattr(row,"AcresBurnedDisplay")=="" : # incomplete data entry 
                        print("WARNING: Incomplete data entry at ", getattr(row,"Name"), "in", getattr(row,"CountiesList"),'on', getattr(row,"StartedDate"), " acres burned :", "NULL")
                        county_acres_burnt[county]=0
                    else :
                        county_acres_burnt[county]=county_acres_burnt[county]+float(getattr(row,"AcresBurnedDisplay").replace(',', ''))
print(county_acres_burnt)

{'Yuba': 11910.0, 'Sierra': 915.0, 'Modoc': 179192.0, 'Marin': 349.0, 'Solano': 63491.393162642424, 'Sonoma': 167636.8421785253, 'Lake': 357711.93414023606, 'Colusa': 75024.7889413172, 'Napa': 187524.75179954732, 'Kern': 122204.04952380952, 'Ventura': 333444.771744363, 'Shasta': 473635.6212251941, 'Trinity': 322012.68938740296, 'Santa Barbara': 245958.20816950634, 'Los Angeles': 164300.89357735976, 'Mono': 32111.455026455027, 'Inyo': 0, 'Tehama': 3228.0, 'Butte': 190702.0, 'El Dorado': 117050.9953482231, 'Sacramento': 1880.8458660680883, 'Amador': 3140.4407294832827, 'Placer': 33877.21573033708, 'Fresno': 214411.0, 'Yolo': 70373.89619965968, 'San Luis Obispo': 98731.16017964072, 'Monterey': 155526.41991017965, 'Kings': 54377.0, 'San Diego': 184.0, 'Nevada': 5366.0, 'Plumas': 63817.0, 'Riverside': 841.0, 'Santa Clara': 7544.0, 'Contra Costa': 6884.0, 'Orange': 36764.0, 'Mariposa': 217852.0, 'Stanislaus': 11283.0, 'Glenn': 301327.45454545453, 'Mendocino': 509600.9976630844, 'Madera': 445